# Investigate InceptionV3 feature extraction

Here we compare tensorflow and keras implementations of trained inception_v3 feature extractor.
The result of both is will be the 2048 feature vector from the last conv layer.

## Questions
* Why does Keras give different results in different sessions? E.g. a bug?
* Why tf.hub and keras.applications feature vectors differ?
* What is the pooling used in the tensorflow model? In Keras we use max pooling.



In [1]:
import numpy as np

import tensorflow as tf
import tensorflow.keras as keras 
import tensorflow_hub as hub
from tensorflow.python.client import device_lib

tf.reset_default_graph()
#tf.set_random_seed(1234)


print(tf.__version__)
print(keras.__version__)
devices = [x.name for x in device_lib.list_local_devices()]
print(devices)


batch_size = 5

batch = np.zeros(shape=(batch_size,299,299,3),dtype=np.float32)
for i in range(batch_size):
    batch[i] = batch[i] + i

W0424 13:52:26.264181 10372 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


1.13.1
2.2.4-tf
['/device:CPU:0', '/device:GPU:0']


In [2]:
with tf.name_scope("Inputs"):
    images = tf.placeholder(shape=(None,299,299,3),dtype=tf.float32)

with tf.name_scope("Tensorflow"):
    tf_inception_v3 = hub.Module("https://tfhub.dev/google/imagenet/inception_v3/feature_vector/1")
    tf_features = tf_inception_v3(images)

with tf.name_scope("Keras"):
    keras_inception_v3 = keras.applications.inception_v3.InceptionV3(include_top=False, weights='imagenet', input_shape=(299,299,3), pooling='max')
    keras_features = keras_inception_v3(images)


for device in devices:

    with tf.device(device):
        
        print('Using device: ',device)
        for i in range(3):
            print('Session ',i)
            with tf.Session() as sess:
                sess.run(tf.global_variables_initializer())
                
                print('first run')
                tf_val,keras_val = sess.run([tf_features, keras_features],feed_dict={images:batch})
                for b in range(batch_size):
                    print('image{}: tf={}  keras={}'.format(b,np.sum(tf_val[b]),np.sum(keras_val[b])))
                    
                print('second run')
                tf_val,keras_val = sess.run([tf_features, keras_features],feed_dict={images:batch})
                for b in range(batch_size):
                    print('image{}: tf={}  keras={}'.format(b,np.sum(tf_val[b]),np.sum(keras_val[b])))
                print('')

Instructions for updating:
Colocations handled automatically by placer.


W0424 13:52:27.989439 10372 deprecation.py:323] From c:\users\i040924\git\notebooks\.venv\lib\site-packages\tensorflow\python\ops\control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0424 13:52:30.604980 10372 saver.py:1483] Saver not created because there are no variables in the graph to restore


Using device:  /device:CPU:0
Session  0
first run
image0: tf=204.5305938720703  keras=0.0
image1: tf=200.66299438476562  keras=0.8595106601715088
image2: tf=379.9951171875  keras=1.7190213203430176
image3: tf=577.4243774414062  keras=2.578531503677368
image4: tf=1049.0826416015625  keras=3.438042640686035
second run
image0: tf=204.5305938720703  keras=0.0
image1: tf=200.66299438476562  keras=0.8595106601715088
image2: tf=379.9951171875  keras=1.7190213203430176
image3: tf=577.4243774414062  keras=2.578531503677368
image4: tf=1049.0826416015625  keras=3.438042640686035

Session  1
first run
image0: tf=204.5305938720703  keras=0.0
image1: tf=200.66299438476562  keras=1.2838647365570068
image2: tf=379.9951171875  keras=2.5677294731140137
image3: tf=577.4243774414062  keras=3.8515944480895996
image4: tf=1049.0826416015625  keras=5.135458946228027
second run
image0: tf=204.5305938720703  keras=0.0
image1: tf=200.66299438476562  keras=1.2838647365570068
image2: tf=379.9951171875  keras=2.567

## How to fix it?
You can load the Keras model weights within the scope of an existing session. It appears that the keras model weights are materialized into the existing session as soon as the model is created.

In [3]:
for device in devices:

    with tf.device(device):
        
        print('Using device: ',device)
        for i in range(3):
            print('Session ',i)
            with tf.Session() as sess:
                tf.keras.backend.set_session(sess)
                keras_inception_v3 = keras.applications.inception_v3.InceptionV3(include_top=False, weights='imagenet', input_shape=(299,299,3), pooling='max')
                keras_features = keras_inception_v3(images)
                #sess.run(tf.global_variables_initializer())
                
                print('first run')
                keras_val = sess.run(keras_features,feed_dict={images:batch})
                for b in range(batch_size):
                    print('image{}: keras={}'.format(b,np.sum(keras_val[b])))
                    
                print('second run')
                keras_val = sess.run(keras_features,feed_dict={images:batch})
                for b in range(batch_size):
                    print('image{}: keras={}'.format(b,np.sum(keras_val[b])))
                print('')

Using device:  /device:CPU:0
Session  0
first run
image0: keras=1059.464599609375
image1: keras=1179.5281982421875
image2: keras=1332.2630615234375
image3: keras=1777.465087890625
image4: keras=2174.394775390625
second run
image0: keras=1059.464599609375
image1: keras=1179.5281982421875
image2: keras=1332.2630615234375
image3: keras=1777.465087890625
image4: keras=2174.394775390625

Session  1
first run
image0: keras=1059.464599609375
image1: keras=1179.5281982421875
image2: keras=1332.2630615234375
image3: keras=1777.465087890625
image4: keras=2174.394775390625
second run
image0: keras=1059.464599609375
image1: keras=1179.5281982421875
image2: keras=1332.2630615234375
image3: keras=1777.465087890625
image4: keras=2174.394775390625

Session  2
first run
image0: keras=1059.464599609375
image1: keras=1179.5281982421875
image2: keras=1332.2630615234375
image3: keras=1777.465087890625
image4: keras=2174.394775390625
second run
image0: keras=1059.464599609375
image1: keras=1179.528198242187

## Observations
### Keras inconsistency
Looks like tensorflow model is consistent across all devices and sessions, whereas the keras one is consistent only within a single session. The issue is related to variable initialization, because when we fixed the rng (tf.set_random_seed(1234)) Keras also becomes consistent. As pointed out by taehoonlee in the [corresponding issue](https://github.com/keras-team/keras-applications/issues/59), this is due to the way Keras initializes model weights (eagerly in the current session). 

### What is the pooling used in both models and why vectors differ?
Looking at the tensorflow graph, the pooling operation seems like tf.reduce_mean (average pooling) with keepdims=true and axis/reduction_indices=(1,2). This makes a ?,8,8,2048 -> ?,1,1,2048 reduction. It may explain the difference in the magnitude of the output values (Keras as configured uses max pooling).

Note: 2048 is the number of filter in the last conv layer (I guess). And 8x8 is the size of the "spatial" fature maps of the penultimate conv layer before pooling
